# Imports

In [14]:
import pandas as pd
from difflib import SequenceMatcher

# 1. 1st Test
- filter gathering related data
- get nouns 
- check frequencies

## 1.1 Get Tweets Related To Gatherings

In [ ]:
with open('words.txt', 'r', encoding='utf-8') as f:
    hotspot_words = f.read().split('\n')

In [ ]:
df = pd.read_csv('all.csv', header=0)

In [ ]:
def hotspot_found(text):
    words = text.split()
    for hotspot_word in hotspot_words:
        for word in words:
            if SequenceMatcher(None, word.lower(), hotspot_word.lower()).ratio() >= 0.8:
                return True
    return False

In [ ]:
hotspots_tweets_mask = df.translated_text.apply(hotspot_found)

In [ ]:
hotspots_tweets_mask.value_counts()

In [ ]:
for x in df[hotspots_tweets_mask].translated_text[:1000]:
    print(x)
    print('------------------------------------')

In [ ]:
df[hotspots_tweets_mask].to_csv('hotspots_tweets_v1.csv', index=False)

## 1.2 Get sentences Nouns to filter non location related words

In [ ]:
df = pd.read_csv('hotspots_tweets_v1.csv', header=0)

In [ ]:
from textblob import TextBlob
df['nouns'] = df.translated_text.apply(lambda text: TextBlob(text).noun_phrases)

In [ ]:
df.head()

In [ ]:
df.to_csv('hotspots_tweets_v2.csv')

## 1.3 Filter non related nouns

In [ ]:
df = pd.read_csv('hotspots_tweets_v2.csv', header=0)

In [ ]:
nouns = df.nouns.str.replace("[", "").str.replace("]", "").str.split(',').explode().reset_index(drop=True)
nouns

In [ ]:
filter_keywords = ['corona', 'twitter', 'http', 'pic', 'covid_19', 'god', 'covid19', 'd8', 'd9', 'lebanon', 'lebanese', 'covid', 'iran', 'house', 'stayhome', 'medical', 'healthcare', 'ministry', 'syrian', 'home', 'china', 
                  'your', 'syria', 'jordan', 'cases']

first = 1
for keyword in filter_keywords:
    if first:
        filter_condition = nouns.str.contains(keyword)
        first = 0
    else:
        filter_condition |= nouns.str.contains(keyword) 

filter1 = filter_condition
filter1.value_counts()

In [ ]:
filtered_nouns = nouns[~filter1].reset_index(drop=True) # `~` is negation

In [ ]:
filtered_nouns

## 1.4 Frequency

In [ ]:
frequencies = filtered_nouns.value_counts()
frequencies

In [ ]:
nouns_df = pd.DataFrame()
nouns_df['noun'] = filtered_nouns
nouns_df['frequency'] = filtered_nouns.replace(frequencies)
nouns_df

In [15]:
nouns_df[nouns_df.frequency > 100]

,noun,frequency
39,'hamad_hassan20',241
52,'hamad_hassan20',241
76,'saudi arabia',190
94,'hamad_hassan20',241
100,'hezbollah',217
...,...,...
121704,'thank',200
121761,'pcr',136
121766,'beirut',714
121815,'aubmc_official',155


## 1.5 Conclusion
- freauencies are not helpful

# 2. Check keywords (I don't see a future for this ;-;)

In [ ]:
df = pd.read_csv('all.csv', header=0)

In [ ]:
def get_stats(keyword):
    data = df[df.translated_text.str.contains(keyword)].translated_text
    print('`', keyword, '` was found in ', len(data), ' rows')
    print('------------------------------------')
    for d in data:
        print(d)
        print('------------------------------------')

In [ ]:
get_stats('university')

In [ ]:
get_stats('exam')

In [ ]:
get_stats('prison')

In [ ]:
get_stats('shop')

In [ ]:
get_stats('store')

In [ ]:
get_stats('gathering')

In [ ]:
get_stats('eid')

In [ ]:
get_stats('festival')

In [ ]:
get_stats('party')

In [ ]:
get_stats('marriage')

# 3. Check locations

In [ ]:
with open('location_related_words.txt', 'r', encoding='utf-8') as f:
    location_words = f.read().split('\n')
location_words

In [ ]:
def accepted(phrase):
    word = None
    for w in location_words:
        if w in phrase:
            word = w
            break
    return pd.Series([phrase, word])

filtered_nouns = pd.DataFrame()
filtered_nouns[['phrase', 'location_word']] = nouns.apply(accepted)
filtered_nouns

In [ ]:
a = filtered_nouns[filtered_nouns['location_word'] != 'None']
a